<a href="https://colab.research.google.com/github/ramziourari/deep-tsundoku/blob/master/notebooks/Donut_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 5.3 MB/s 
     |████████████████████████████████| 7.0 MB 43.2 MB/s 


In [1]:
!pip install gradio --quiet

     |████████████████████████████████| 5.3 MB 5.4 MB/s 
     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 270 kB 42.6 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 212 kB 55.6 MB/s 
     |████████████████████████████████| 112 kB 62.4 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 2.3 MB 53.8 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 9.0 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 35.7 MB/s 
     |████████████████████████████████| 594 kB 38.6 MB/s 
     |████████████████████████████████| 856 kB 62.9 MB/s 


In [3]:
import torch
from transformers import DonutProcessor, VisionEncoderDecoderModel
import gradio as gr
import re

In [4]:
processor = DonutProcessor.from_pretrained("jay-jojo-cheng/donut-cover")
model = VisionEncoderDecoderModel.from_pretrained("jay-jojo-cheng/donut-cover")

Downloading:   0%|          | 0.00/362 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/355 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
def predict_from_spine(img):

  pixel_values = processor(img, return_tensors="pt").pixel_values
  
  task_prompt = "<s_cord-v2>"
  decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)

  outputs = model.generate(pixel_values.to(device),
                            decoder_input_ids=decoder_input_ids.to(device),
                            max_length=model.decoder.config.max_position_embeddings,
                            early_stopping=True,
                            pad_token_id=processor.tokenizer.pad_token_id,
                            eos_token_id=processor.tokenizer.eos_token_id,
                            use_cache=True,
                            num_beams=1,
                            bad_words_ids=[[processor.tokenizer.unk_token_id]],
                            return_dict_in_generate=True,
                            output_scores=True,)
  sequence = processor.batch_decode(outputs.sequences)[0]
  sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
  return sequence

In [7]:
gr.Interface(fn=predict_from_spine, 
             inputs=gr.Image(type="pil"),
             outputs=gr.components.Textbox(),
             examples=["/content/001.jpg", "/content/002.jpg", "/content/003.jpg"]).launch(share=True, show_error=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://19876.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f20ff059350>,
 'http://127.0.0.1:7860/',
 'https://19876.gradio.app')